In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_excel(r'C:\Users\sanla\Desktop\Prediction of CO Conversion\Dataset_Intern.xlsx')
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [4]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [5]:
from lightgbm import LGBMRegressor
regressor = LGBMRegressor()
regressor.fit(x_train, y_train)

LGBMRegressor()

In [6]:
y_pred = regressor.predict(x_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[56.55 89.8 ]
 [57.97 99.4 ]
 [33.18 24.2 ]
 ...
 [11.91 20.6 ]
 [82.38  4.71]
 [26.94 22.4 ]]


In [7]:
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
score = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(score)
print(mse)

0.6639328385979548
362.57352002448783


In [8]:
from sklearn.model_selection import cross_val_score
scorer = make_scorer(mean_squared_error)
scores = cross_val_score(estimator = regressor, X = x_train, y = y_train, scoring = 'r2', cv = 10)
amse = cross_val_score(estimator = regressor, X = x_train, y = y_train, scoring = scorer, cv = 10)
print(scores.mean() * 100)
print(scores.std() * 100)
print(amse.mean())

79.82927956106633
3.230659239379974
209.5491676966952


In [9]:
from sklearn.model_selection import GridSearchCV
parameters = [{
    'learning_rate': [0.1],
    'n_estimators': [328],
    'max_depth': [None],
    'num_leaves': [32],
    'min_child_samples': [11],
    'colsample_bytree': [0.6],
    'subsample' : [0.4],
    'min_split_gain' : [0.0],
    'reg_alpha': [0.1],
    'reg_lambda': [0.1],
}]
grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           scoring = 'r2',
                          cv = 15,
                          n_jobs = -1)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=15, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid=[{'colsample_bytree': [0.6], 'learning_rate': [0.1],
                          'max_depth': [None], 'min_child_samples': [11],
                          'min_split_gain': [0.0], 'n_estimators': [328],
                          'num_leaves': [32], 'reg_alpha': [0.1],
                          'reg_lambda': [0.1], 'subsample': [0.4]}],
             scoring='r2')

In [10]:
print(grid_search.best_score_)

0.8284206678337083


In [11]:
print(grid_search.best_params_)

{'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': None, 'min_child_samples': 11, 'min_split_gain': 0.0, 'n_estimators': 328, 'num_leaves': 32, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.4}


In [12]:
amse = cross_val_score(estimator = grid_search, X = x_train, y = y_train, scoring = scorer, cv = 15)
print(amse.mean())

177.0934363259865
